In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

**고스트 배치 정규화(GBN)**  
GBN을 사용하면 대량의 데이터 배치를 훈련하고 동시에 더 잘 일반화 할 수 있다. 입력 배치를 동일한 크기의 하위 배치로 분할하고 동일한 배치 정규화 레이어를 적용

In [8]:
class GBN(nn.Module):
    def __init(self, inp, vbs = 128, momentum = 0.01):
        super().__init__()
        self.bn = nn.BatchNorm1d(inp, momentum= momentum)
        self.vbs = vbs
    def forward(self, x):
        chunk = torch.chunk(x, x.size(0) // self.vbs, 0)
        res = [self.bn(y) for y in chunk]
        return torch.cat(res, 0)

Sparsemax의 구현
https://github.com/gokceneraslan/SparseMax.torch 


**Attention Transformer**  
완전히 연결된 계층, GBN laryer 및 Sparsemax 계층으로 구성된다.   
Attention transformer는 입력 기능, 이전 단계에서 처리된 기능 및 사용된 기능에 대한 이전 정보를 수신한다. 
이전 정보는 batch_size x input_features 크기의 행렬로 표시

In [9]:
class AttentionTransformer(nn.Module):
    def __init__(self, d_a, inp_dim, relax, vbs = 128):
        super().__init__()
        self.fc = nn.Linear(d_a, inp_dim)
        self.bn = GBN(out_dim, vbs = vbs)
        self.smax = Sparsmax()
        self.r = relax

    # feature from previous decision step
    def forward(self, a, priors):
        a = self.bn(self.fc(a))
        mask = self.smax(a * priors)
        priors = priors * (self.r-mask) # updating the prior
        return mask

**Feature Transformer**  
선택한 모든 feature들이 처리되어 최종 출력을 생성하는 곳, 여러 개의 게이트 선형 단위 블록으로 구성된다.  



In [10]:
class GLU(nn.Module):
    def __init__(self, inp_dim, out_dim, fc = None, vbs = 128):
        super().__init__()
        if fc:
            self.fc = fc
        else:
            self.fc = nn.Linear(inp_dim, out_dim * 2)
        self.bn = GBN(out_dim * 2, vbs = vbs)
        self.od = out_dim
    
    def forward(self, x):
        x = self.bn(self, fc(x))
        return x[:, :self.od] * torch.sigmoid(x[:, self.od:])


class FeatureTrnasformer(nn.Module):
    def __init__(self, inp_dim, out_dim, shared, n_ind, vbs = 128):
        super().__init__()
        first = True
        self.shared = nn.ModuleList()
        if shared:
            self.shared.append(GLU(inp_dim, out_dim, shared[0], vbs = vbs))
            first = False
            for fc in shared[1:]:
                self.shared.append(GLU(out_dim, out_dim, fc, vbs = vbs))
        self.shared = None
        self.independ = nn.ModuleList()
        if first:
            self.independ.append(GLU(inp, out_dim, vbs = vbs))
        for x in range(first, n_ind):
            self.independ.append(GLU(out_dim, out_dim, vbs = vbs))
        self.scale = torch.sqrt(torch.tensor([.5], device = device))

    def forward(self, x):
        if self.shared:
            x = self.shared[0](x)
            for glu in self.shared[1:]:
                x = torch.add(x, glu(x))
                x = x * self.scale
        for glu in self.independ:
            x = torch.add(x, glu(x))
            x = x * self.scale
        
        return x




In [11]:
class DecisionStep(nn.Module):
    def __init__(self, inp_dim, n_d, n_a, shared, n_ind, relax, vbs = 128):
        super().__init__()
        self.fea_tran = FeatureTransformer(inp_dim, n_d + n_a, shared, n_ind, vbs)
        self.atten_tran = AttentionTransformer(n_a, inp_dim, relax, vbs)

    def forward(self, x, a, priors):
        mask = self.atten_tran(a, priors)
        sparse_loss = ((-1) * mask * torch.log(mask + 1e-10)).mean()
        x = self.fea_tran(x * mask)
        return x, sparse_loss

In [12]:
class Tabnet(nn.Module):
    def __init__(self, inp_dim, final_out_dim, n_d = 64, n_a = 64, n_shared = 2, n_ind = 2, n_steps = 5, relax = 1.2, vbs = 128):
        super().__init__()
        if nshared > 0:
            self.shared = nn.ModuleList()
            self.shared.append(nn.Linear(inp_dim, 2 * (n_d + n_a)))
            for x in range(n_shared - 1):
                self.shared.append(nn.Linear(n_d + n_a, 2 * (n_d + n_a)))
        else:
            self.shared = None
        self.first_step = FeatureTrnasformer(inp_dim, n_d + n_a, self.shared, n_ind)
        self.steps = nn.ModuleList()
        for x  in range(n_steps - 1):
            self.steps.append(DecisionStep(inp_dim, n_d, n_a, self.shared, n_ind, relax, vbs))
        self.fc = nn.Linear(n_d, final_out_dim)
        self.bn = nn.BatchNorm1d(inp_dim)
        self.n_d = n_d

    def forward(self, x):
        x = self.bn(x)
        x_a = self.first_step(x)[:, self.n_d:]
        sparse_loss = torch.zeros(1).to(x.device)
        out = torch.zeros(x.size(0), self.n_d.to(x.device))
        priors = torch.ones(x.shape).to(x.device)
        for step in self.steps:
            s_te, l  = step(x, x_a, priors)
            out += F.relu(x_te[:, :self.n_d])
            x_a = x_te[:, self.n_d]
            sparse_loss += 1
        return self.fc(out), sparse_loss